## Train and compare the performance of residential location choice models

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from functions import set_seed, train, evaluate_nn, Config
from models import MNL_Choice, SCL_Choice, GNNChoiceModel, MLP_Choice
from data_process import load_data, spatial_choice_dataset_interact

comm, hh, edge_index, distance_to_work = load_data()

comm_features = [
    "pop_density",
    "white_prop",
    "black_prop",
    "single_res",
    "multi_res",
    "office",
    # "retail",
    "land_mix",
    "transit_a_scaled",
    "med_house_age_scaled",
    "med_value_scaled",
    "h_units_scaled",
    "median_inc_scaled",
]

hh_features = ["hh_income_scaled", "race_white", "race_black"]

### GNN 1 layer

In [2]:
# Set the training and evaluation procedure
device = torch.device("cpu")
config = Config()
config.bs = 32
config.num_hidden = 64
config.dropout = 0.05
config.optimizer = "adam"  # one of [adam, sgd]
config.lr = 0.01
config.lr_scheduler = "one_cycle"  # one of [step, one_cycle, exp, none]
config.n_epoch = 20
config.model = "GATConv"  # or any other model name
config.heads = 4  # Number of attention heads for GAT
config.mode = "disabled"  # online or disabled
config.residual = True
config.seed = 100
config.n_layer = 1  # Number of layers for GNN

my_dataset = spatial_choice_dataset_interact
train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.bs, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = GNNChoiceModel(
        train_dataset[0][0].shape[-1],
        config.num_hidden,
        edge_index,
        dropout=config.dropout,
        heads=config.heads,
        residual=config.residual,
        n_layer=config.n_layer,
    ).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")

LLL_train: -11685.1367 	 LLL_test: -1348.8076 	         accuracy_train: 0.1355 	 accuracy_test: 0.1068 	         Fold: 1/10
LLL_train: -11681.5215 	 LLL_test: -1342.5398 	         accuracy_train: 0.1338 	 accuracy_test: 0.1016 	         Fold: 2/10
LLL_train: -11718.0693 	 LLL_test: -1335.9983 	         accuracy_train: 0.1268 	 accuracy_test: 0.1406 	         Fold: 3/10
LLL_train: -11718.1016 	 LLL_test: -1320.6229 	         accuracy_train: 0.1361 	 accuracy_test: 0.1328 	         Fold: 4/10
LLL_train: -11749.6934 	 LLL_test: -1289.4059 	         accuracy_train: 0.1262 	 accuracy_test: 0.1693 	         Fold: 5/10
LLL_train: -11697.5859 	 LLL_test: -1335.7532 	         accuracy_train: 0.1332 	 accuracy_test: 0.1146 	         Fold: 6/10
LLL_train: -11723.0879 	 LLL_test: -1314.1182 	         accuracy_train: 0.1335 	 accuracy_test: 0.1094 	         Fold: 7/10
LLL_train: -11784.6318 	 LLL_test: -1277.8662 	         accuracy_train: 0.1285 	 accuracy_test: 0.1562 	         Fold: 8/10
LLL_trai

### GNN 2 layers

In [3]:
# Set the training and evaluation procedure
device = torch.device("cpu")
config = Config()
config.bs = 32
config.num_hidden = 64
config.dropout = 0.05
config.optimizer = "adam"  # one of [adam, sgd]
config.lr = 0.01
config.lr_scheduler = "one_cycle"  # one of [step, one_cycle, exp, none]
config.n_epoch = 20
config.model = "GATConv"  # or any other model name
config.heads = 4  # Number of attention heads for GAT
config.mode = "disabled"  # online or disabled
config.residual = True
config.seed = 100
config.n_layer = 2  # Number of layers for GNN

my_dataset = spatial_choice_dataset_interact
train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.bs, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = GNNChoiceModel(
        train_dataset[0][0].shape[-1],
        config.num_hidden,
        edge_index,
        dropout=config.dropout,
        heads=config.heads,
        residual=config.residual,
        n_layer=config.n_layer,
    ).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")


LLL_train: -11553.3252 	 LLL_test: -1345.5552 	         accuracy_train: 0.1395 	 accuracy_test: 0.1068 	         Fold: 1/10
LLL_train: -11563.1836 	 LLL_test: -1331.7244 	         accuracy_train: 0.1378 	 accuracy_test: 0.1016 	         Fold: 2/10
LLL_train: -11579.7842 	 LLL_test: -1323.5992 	         accuracy_train: 0.1309 	 accuracy_test: 0.1380 	         Fold: 3/10
LLL_train: -11565.9336 	 LLL_test: -1309.3973 	         accuracy_train: 0.1398 	 accuracy_test: 0.1406 	         Fold: 4/10
LLL_train: -11567.4922 	 LLL_test: -1287.2847 	         accuracy_train: 0.1343 	 accuracy_test: 0.1615 	         Fold: 5/10
LLL_train: -11558.4277 	 LLL_test: -1325.2070 	         accuracy_train: 0.1369 	 accuracy_test: 0.1094 	         Fold: 6/10
LLL_train: -11612.0537 	 LLL_test: -1310.4956 	         accuracy_train: 0.1367 	 accuracy_test: 0.1094 	         Fold: 7/10
LLL_train: -11643.6846 	 LLL_test: -1264.2719 	         accuracy_train: 0.1323 	 accuracy_test: 0.1667 	         Fold: 8/10
LLL_trai

### GNN 3 Layers

In [4]:
# Set the training and evaluation procedure
device = torch.device("cpu")
config = Config()
config.bs = 32
config.num_hidden = 64
config.dropout = 0.05
config.optimizer = "adam"  # one of [adam, sgd]
config.lr = 0.01
config.lr_scheduler = "one_cycle"  # one of [step, one_cycle, exp, none]
config.n_epoch = 20
config.model = "GATConv"  # or any other model name
config.heads = 4  # Number of attention heads for GAT
config.mode = "disabled"  # online or disabled
config.residual = True
config.seed = 100
config.n_layer = 3  # Number of layers for GNN

my_dataset = spatial_choice_dataset_interact
train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.bs, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = GNNChoiceModel(
        train_dataset[0][0].shape[-1],
        config.num_hidden,
        edge_index,
        dropout=config.dropout,
        heads=config.heads,
        residual=config.residual,
        n_layer=config.n_layer,
    ).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")


LLL_train: -11542.4424 	 LLL_test: -1343.0680 	         accuracy_train: 0.1375 	 accuracy_test: 0.0938 	         Fold: 1/10
LLL_train: -11574.4238 	 LLL_test: -1333.5273 	         accuracy_train: 0.1387 	 accuracy_test: 0.1068 	         Fold: 2/10
LLL_train: -11581.3906 	 LLL_test: -1329.1145 	         accuracy_train: 0.1320 	 accuracy_test: 0.1432 	         Fold: 3/10
LLL_train: -11572.8730 	 LLL_test: -1314.2314 	         accuracy_train: 0.1361 	 accuracy_test: 0.1302 	         Fold: 4/10
LLL_train: -11610.1484 	 LLL_test: -1285.1687 	         accuracy_train: 0.1323 	 accuracy_test: 0.1615 	         Fold: 5/10
LLL_train: -11590.4092 	 LLL_test: -1327.3518 	         accuracy_train: 0.1381 	 accuracy_test: 0.1172 	         Fold: 6/10
LLL_train: -11564.2891 	 LLL_test: -1308.7520 	         accuracy_train: 0.1393 	 accuracy_test: 0.1146 	         Fold: 7/10
LLL_train: -11608.0898 	 LLL_test: -1266.5388 	         accuracy_train: 0.1326 	 accuracy_test: 0.1615 	         Fold: 8/10
LLL_trai

### MLP model

In [5]:
# Set the training and evaluation procedure
device = torch.device("cpu")
config = Config()
config.bs = 32
config.num_hidden = 64
config.dropout = 0
config.optimizer = "adam"  # one of [adam, sgd]
config.lr = 0.01
config.lr_scheduler = "one_cycle"  # one of [step, one_cycle, exp, none]
config.n_epoch = 20
config.model = "MLP"  # or any other model name
config.mode = "disabled"  # online or disabled
config.seed = 100
my_dataset = spatial_choice_dataset_interact

train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config.bs, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = MLP_Choice(
        train_dataset[0][0].shape[-1], config.num_hidden, config.dropout
    ).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")


LLL_train: -11769.1465 	 LLL_test: -1349.9144 	         accuracy_train: 0.1277 	 accuracy_test: 0.1250 	         Fold: 1/10
LLL_train: -11756.4160 	 LLL_test: -1351.1594 	         accuracy_train: 0.1320 	 accuracy_test: 0.0990 	         Fold: 2/10
LLL_train: -11784.2598 	 LLL_test: -1336.1099 	         accuracy_train: 0.1291 	 accuracy_test: 0.1354 	         Fold: 3/10
LLL_train: -11800.1299 	 LLL_test: -1324.1406 	         accuracy_train: 0.1300 	 accuracy_test: 0.1354 	         Fold: 4/10
LLL_train: -11816.6611 	 LLL_test: -1296.4132 	         accuracy_train: 0.1245 	 accuracy_test: 0.1641 	         Fold: 5/10
LLL_train: -11760.4424 	 LLL_test: -1337.9064 	         accuracy_train: 0.1323 	 accuracy_test: 0.1042 	         Fold: 6/10
LLL_train: -11764.4883 	 LLL_test: -1320.2679 	         accuracy_train: 0.1309 	 accuracy_test: 0.1016 	         Fold: 7/10
LLL_train: -11817.2031 	 LLL_test: -1275.3146 	         accuracy_train: 0.1274 	 accuracy_test: 0.1536 	         Fold: 8/10
LLL_trai

### SCL model

In [6]:
# Set the training and evaluation procedure
config = Config()
config.num_hidden = 64
config.dropout = 0
config.optimizer = "lbfgs"  # one of [adam, sgd]
config.lr = 0.1
config.n_epoch = 20
config.model = "SCL"  # or any other model name
config.mode = "disabled"  # online or disabled
config.seed = 100
my_dataset = spatial_choice_dataset_interact

train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=len(train_dataset), shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = SCL_Choice(train_dataset[0][0].shape[-1], edge_index).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results
    print(f"Estimated mu value={torch.sigmoid(model.mu_raw).detach().numpy()}")

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")


LLL_train: -11981.8516 	 LLL_test: -1360.5005 	         accuracy_train: 0.1233 	 accuracy_test: 0.1094 	         Fold: 1/10
Estimated mu value=0.9999557733535767
LLL_train: -11996.9873 	 LLL_test: -1344.1005 	         accuracy_train: 0.1239 	 accuracy_test: 0.1094 	         Fold: 2/10
Estimated mu value=0.9999487400054932
LLL_train: -11975.9424 	 LLL_test: -1366.6660 	         accuracy_train: 0.1248 	 accuracy_test: 0.1224 	         Fold: 3/10
Estimated mu value=0.9999711513519287
LLL_train: -11988.4316 	 LLL_test: -1354.8373 	         accuracy_train: 0.1257 	 accuracy_test: 0.1224 	         Fold: 4/10
Estimated mu value=0.9999268054962158
LLL_train: -12026.8545 	 LLL_test: -1315.6639 	         accuracy_train: 0.1184 	 accuracy_test: 0.1484 	         Fold: 5/10
Estimated mu value=0.9999346733093262
LLL_train: -11981.1885 	 LLL_test: -1360.4240 	         accuracy_train: 0.1265 	 accuracy_test: 0.1016 	         Fold: 6/10
Estimated mu value=0.9999693632125854
LLL_train: -11995.2080 	 LLL

### MNL model

In [7]:
# Set the training and evaluation procedure
config = Config()
config.num_hidden = 64
config.dropout = 0
config.optimizer = "lbfgs"  # one of [adam, sgd]
config.lr = 0.1
config.n_epoch = 20
config.model = "MNL"  # or any other model name
config.mode = "disabled"  # online or disabled
config.seed = 100
my_dataset = spatial_choice_dataset_interact

train_results = []
test_results = []
for i in range(10):
    test_idx = np.arange(i, len(hh), 10)
    train_idx = np.setdiff1d(np.arange(len(hh)), test_idx)
    train_dataset = my_dataset(
        comm,
        hh.loc[train_idx, :],
        distance_to_work[train_idx],
        comm_features,
        hh_features,
    )
    test_dataset = my_dataset(
        comm,
        hh.loc[test_idx, :],
        distance_to_work[test_idx],
        comm_features,
        hh_features,
    )

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=len(train_dataset), shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=len(test_dataset), shuffle=False
    )
    criterion = nn.NLLLoss()

    set_seed(config.seed)
    model = MNL_Choice(train_dataset[0][0].shape[-1],
                       train_dataset[0][0].shape[0]).to(device)

    set_seed(config.seed)
    model = train(
        model,
        criterion,
        train_loader,
        None,
        test_loader,
        config,
        device,
        verbose=False,
    )
    train_results.append(evaluate_nn(model, train_loader, comm))
    test_results.append(evaluate_nn(model, test_loader, comm))

    print(
        f"LLL_train: {train_results[-1]['LLL']:.4f} \t LLL_test: {test_results[-1]['LLL']:.4f} \t \
        accuracy_train: {train_results[-1]['accuracy']:.4f} \t accuracy_test: {test_results[-1]['accuracy']:.4f} \t \
        Fold: {i + 1}/10"
    )  # Log the results

train_results = {
    key: [d[key] for d in train_results] for key in train_results[0].keys()
}
test_results = {key: [d[key] for d in test_results] for key in test_results[0].keys()}
train_results = pd.DataFrame(train_results)
test_results = pd.DataFrame(test_results)
train_results.to_csv(f"results/{config.model}_train_results.csv", index=False)
test_results.to_csv(f"results/{config.model}_test_results.csv", index=False)

print(f"Train results mean: \n{train_results.mean()}")
print(f"Test results mean: \n{test_results.mean()}")


LLL_train: -11981.8496 	 LLL_test: -1360.5159 	         accuracy_train: 0.1233 	 accuracy_test: 0.1094 	         Fold: 1/10
LLL_train: -11996.9863 	 LLL_test: -1344.0975 	         accuracy_train: 0.1236 	 accuracy_test: 0.1094 	         Fold: 2/10
LLL_train: -11975.9414 	 LLL_test: -1366.6653 	         accuracy_train: 0.1248 	 accuracy_test: 0.1224 	         Fold: 3/10
LLL_train: -11988.4287 	 LLL_test: -1354.8384 	         accuracy_train: 0.1257 	 accuracy_test: 0.1224 	         Fold: 4/10
LLL_train: -12026.8516 	 LLL_test: -1315.6572 	         accuracy_train: 0.1184 	 accuracy_test: 0.1484 	         Fold: 5/10
LLL_train: -11981.1875 	 LLL_test: -1360.4270 	         accuracy_train: 0.1265 	 accuracy_test: 0.1016 	         Fold: 6/10
LLL_train: -11995.2070 	 LLL_test: -1347.7167 	         accuracy_train: 0.1262 	 accuracy_test: 0.1146 	         Fold: 7/10
LLL_train: -12041.0654 	 LLL_test: -1300.9255 	         accuracy_train: 0.1213 	 accuracy_test: 0.1380 	         Fold: 8/10
LLL_trai